In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations

from astropy.table import Table
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker

import SAGA
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord
from SAGA.utils.distance import z2v

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

sats = Query(C.is_sat).filter(base)
del base

In [ ]:
assert len(sats) == 127
assert C.r_abs_limit.count(sats) == 123
assert len(np.unique(sats["HOSTID"])) == 34

In [ ]:
sats = add_skycoord(sats)

In [ ]:
sdss_x = np.linspace(0, 180, 19)[:-1] + 5
sdss_y = np.array([0.64268367, 0.44155338, 0.4841401 , 0.4633828 , 0.4685856 ,
       0.58841093, 0.54086203, 0.64018368, 0.48222288, 0.51515562,
       0.52488057, 0.55817479, 0.51704362, 0.47159272, 0.44232658,
       0.47630996, 0.58469492, 0.49163754])

In [ ]:
X = []

for sats_this in sats.group_by('HOSTID').groups:
    if len(sats_this) < 2:
        continue

    host_this = str(sats_this['HOSTID'][0])
    host_coord = SkyCoord(sats_this['HOST_RA'][0], sats_this['HOST_DEC'][0], unit="deg")
    
    sep_deg = host_coord.position_angle(sats_this['coord']).degree
    dv = z2v(sats_this['SPEC_Z']) - sats_this['HOST_VHOST']
    
    for i, j in combinations(range(len(sats_this)), 2):
        sat1 = sats_this[i]
        sat2 = sats_this[j]
        X.append((host_this, sep_deg[i], sep_deg[j], dv[i], dv[j], sat1["RHOST_KPC"], sat2["RHOST_KPC"], 
                  1000*sat1["HOST_DIST"]*sat1['coord'].separation(sat2['coord']).radian))
    
X = Table(np.array(X, dtype=np.dtype([('HOSTID', '<U12'), ('pa1', np.float), ('pa2', np.float), ('dv1', np.float), ('dv2', np.float), ('d1', np.float), ('d2', np.float), ('sep', np.float)])))

In [ ]:
X["sign"] = np.sign(X["dv1"]) * np.sign(X["dv2"])
dpa = np.abs(X["pa1"] - X["pa2"])
X["opening_angle"] = np.where(dpa < 180, 180-dpa, dpa-180)
X["corotating"] = np.where(X["opening_angle"] < 90, X["sign"] == -1, X["sign"] == 1)

In [ ]:
dv_cut = Query('abs(dv1) > 25', 'abs(dv2) > 25')
#dv_cut = Query()
radius_cut = Query("d1<150", "d2<150")

print(dv_cut.count(X))
print((dv_cut & radius_cut).count(X))
print()
print((~dv_cut).count(X))

In [ ]:
plt.figure(figsize=(4.8,4))

bins = np.linspace(0, 180, 11)
hist_co = np.histogram(Query('corotating', dv_cut).filter(X, 'opening_angle'), bins)[0]
hist_all = np.histogram(dv_cut.filter(X, 'opening_angle'), bins)[0]

p = (hist_co+1)/(hist_all+2)
perr = np.sqrt(p*(1-p) / hist_all)
p1 = np.minimum(1, p + perr)
p2 = np.maximum(0, p - perr)
plt.fill_between((bins[1:]+bins[:-1])*0.5, p1, p2, lw=0, color="C0", alpha=0.2, label="SAGA sats$<300$ kpc");

bins = np.linspace(0, 180, 11)
hist_co_inner = np.histogram(Query('corotating', dv_cut, radius_cut).filter(X, 'opening_angle'), bins)[0]
hist_all_inner = np.histogram(Query(dv_cut, radius_cut).filter(X, 'opening_angle'), bins)[0]

p = (hist_co_inner+1)/(hist_all_inner+2)
perr = np.sqrt(p*(1-p) / hist_all_inner)
p1 = np.minimum(1, p + perr)
p2 = np.maximum(0, p - perr)

plt.errorbar((bins[1:]+bins[:-1])*0.5, p, yerr=[p-p2, p1-p], lw=1, ls="", color='C2', label="SAGA sats$<150$ kpc", marker='s');
plt.errorbar(sdss_x, sdss_y, c="C7", alpha=0.8, label="SDSS (Phillips+ 2015, Fig. 3b)")

plt.text(3, 1.04, r"Opposite side")
plt.text(177, 1.04, r"Same side", ha="right")

plt.legend(fontsize='small', loc="lower left")
plt.xlim(0, 180.0);
plt.xticks([0,30,60,90,120,150,180])
plt.axhline(0.5, c='k', ls='--', lw=0.5);
plt.xlabel('Opening angle [deg]');
plt.ylabel('Co-rotating fraction')
plt.ylim(-0.15, 1.15)
plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
plt.tight_layout()
plt.savefig('/home/yymao/Downloads/sat_coroating_pairs.pdf', bbox_inches='tight')

## additional plots

In [ ]:
fig, ax = plt.subplots(ncols=3, gridspec_kw={'wspace':0.1, 'width_ratios':(30,1,1)})
t = Query('corotating', dv_cut, 'abs(dv1-dv2) > 30').filter(X) 
CS = ax[0].scatter(t["opening_angle"], t["sep"], c=np.abs(t["dv1"]-t["dv2"]), vmin=0, vmax=400, cmap="Blues_r", label="Co-rotating pair")
plt.colorbar(CS, ax[1])

t = Query('corotating == 0', dv_cut, 'abs(dv1-dv2) > 30').filter(X)
CS = ax[0].scatter(t["opening_angle"], t["sep"], c=np.abs(t["dv1"]-t["dv2"]), vmin=0, vmax=400, cmap="Oranges_r", label="Counter-rotating pair")
cbar = plt.colorbar(CS, ax[2])
ax[1].set_yticklabels([])
ax[2].set_xlabel(r"$|\Delta v|$ [km s$^{-1}$]", fontsize='small')

lgnd = ax[0].legend(markerfirst=False, scatterpoints=3, frameon=True)
lgnd.legendHandles[0].set_color(plt.cm.Blues_r([0.2, 0.5, 0.7]))
lgnd.legendHandles[1].set_color(plt.cm.Oranges_r([0.2, 0.5, 0.7]))
ax[0].axvline(90, color="grey", ls='--')
ax[0].set_xlim(-5, 185.0);
ax[0].set_xlabel('Opening angle [deg]');
ax[0].set_ylabel('Projected separation [kpc]')
ax[0].set_ylim(0, 600.0);